In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
print(device)

with open('data/tinyshakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))

d_input = len(chars) + 1 # let 0 be padding
print(d_input)

stoi = lambda c: chars.index(c) + 1
itos = lambda n: "" if n == 0 else chars[n-1]
encode = lambda s: torch.tensor([stoi(c) for c in s], dtype=torch.long)
decode = lambda m: ''.join([itos(i) for i in m])

code = encode('abc')
print(code)
print(decode(code))

data = encode(text)
n_split = int(0.9 * len(data))
train_data = data[:n_split]
test_data = data[n_split:]

def get_batch(mode, seq_len, batch_size=1):
    source = train_data if mode == 'train' else test_data
    starts = torch.randint(len(source) - seq_len - 1, (batch_size, ))
    x = torch.stack([source[s:s+seq_len] for s in starts])
    y = torch.stack([source[s+1:s+1+seq_len] for s in starts])
    x, y = x.to(device), y.to(device)
    return x, y

eval_iters = 50 #200
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for mode in ['train', 'eval']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(mode, max_seq_len)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[mode] = losses.mean()
    model.train()
    return out

cuda
66
tensor([40, 41, 42])
abc


In [10]:
import gpt

max_seq_len = 200

model = gpt.GPT(d_model=32, d_input=d_input, max_seq_len=max_seq_len, N=12)
try:
    PATH = "models/gptv2.pth"
    model.load_state_dict(torch.load(PATH))
    print("loaded model")
except:
    print("new model")

torch.cuda.empty_cache()
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2)

new model


In [12]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("runs/tinygpt")

writer.add_graph(model, get_batch('train', max_seq_len, 1))
writer.close()

In [19]:
max_iters = 300
for step in range(0, max_iters):
    
    x, y = get_batch('train', max_seq_len, 50)

    z, loss = model(x, y)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if step % (max_iters // 10) == 0 or step == max_iters-1:
        losses = estimate_loss()
        print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['eval']:.4f}")

        writer.add_scalar('training loss', losses['train'] / 100, step)
        writer.add_scalar('validation loss', losses['eval'] / 100, step)

OutOfMemoryError: CUDA out of memory. Tried to allocate 62.00 MiB (GPU 0; 6.00 GiB total capacity; 5.25 GiB already allocated; 0 bytes free; 5.26 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [22]:
PATH = "models/gptv1.pth"
torch.save(model.state_dict(), PATH)

In [21]:
context = encode("ANTIGONUS:").unsqueeze(0).to(device) # add batch to 1

result = model.generate(context, new_seq_len=300)[0]
print(decode(result))

ANTIGONUS:


e
u  :


 

,
,  
 


 
   
 
   :, l : 

      r were test Coaten:
So, that's mother: you have more:
And un it an by enemerbetaled's genlastrest.

QUO:
And that,sere sadder!

DUCHESS IV:
I so well at hiss part.
Thou'd which this ground Lecondersain, willing.
To he comeod again allis as thou grag


In [6]:
x = torch.randn(2, 4)
y = torch.zeros(1, 5)

print(F.pad(input=x, pad=(0, 3, 0, 0), mode='constant', value=0).shape)

torch.Size([2, 7])
